In [29]:
import numpy as np
import thermofield_boltz_funcs as bz
import opt_einsum as oe
import scipy.sparse as sp
from quant_rotor.models.sparse.support_ham import build_V_prime_in_p
from quant_rotor.core.dense.de_solve_one_thermal_dense import integration_scheme as integration_scheme_new_dense
from quant_rotor.core.sparse.de_solve_one_thermal_sparse import integration_scheme as integration_scheme_new_sparse
from quant_rotor.core.dense.de_solve_one_thermal import integration_scheme as integration_scheme_old

np.set_printoptions(threshold=np.inf, linewidth=np.inf)

In [30]:
site = 3
state = 3
g = 0.1

In [31]:
beta_d, t_0_dB_d, t_0_sol_d, t_2_max_d  = integration_scheme_new_dense(site, state, g, t_init=0,t_final=10,nof_points=100000)

Done.


In [32]:
beta_s, t_0_dB_s, t_0_sol_s, t_2_max_s  = integration_scheme_new_sparse(site, state, g, t_init=0,t_final=10,nof_points=100000)

Done.


In [33]:
K, V = build_V_prime_in_p(state)
I = np.eye(state)

U, _ = bz.thermofield_change_of_basis(I)

U_sparse = sp.csr_matrix(U)

K_tilda = U_sparse.T @ K @ U_sparse

V_tilda = oe.contract('Mi,Wj,ijab,aN,bV->MWNV', U, U, V.toarray().reshape(state**2, state**2, state**2, state**2), U, U, optimize='optimal')

In [34]:
beta_TF, t_0_dB_TF, t_0_sol_TF, t_2_max_TF  = integration_scheme_old(site, state**2, g, t_init=0,t_final=10,nof_points=100000, import_K_V_TF=True, K_import=K_tilda.toarray(), V_import=V_tilda)

In [35]:
print(np.allclose(beta_s, beta_d))
print(np.allclose(t_0_dB_s, t_0_dB_d))
print(np.allclose(t_0_sol_s, t_0_sol_d))
print(np.allclose(t_2_max_s, t_2_max_d))

True
False
False
False


In [36]:
print(np.allclose(beta_s, beta_TF))
print(np.allclose(t_0_dB_s, t_0_dB_TF))
print(np.allclose(t_0_sol_s, t_0_sol_TF))
print(np.allclose(t_2_max_s, t_2_max_TF))

True
False
False
False


In [37]:
print(np.allclose(beta_d, beta_TF))
print(np.allclose(t_0_dB_d, t_0_dB_TF))
print(np.allclose(t_0_sol_d, t_0_sol_TF))
print(np.allclose(t_2_max_d, t_2_max_TF))

True
True
True
True


# Check the interative step.

In [8]:
comb_flat = np.random.rand(201)

In [9]:
site = 3
state = 3
g = 0.2

In [10]:
K, V = build_V_prime_in_p(state)
I = np.eye(state)

U, _ = bz.thermofield_change_of_basis(I)

U_sparse = sp.csr_matrix(U)

h_full = (U_sparse.T @ K @ U_sparse).toarray()

v_full = oe.contract('Mi,Wj,ijab,aN,bV->MWNV', U, U, V.toarray().reshape(state**2, state**2, state**2, state**2), U, U, optimize='optimal')
v_full = v_full * g

## New.

In [11]:
from quant_rotor.models.dense.t_amplitudes_sub_class_new import QuantumSimulation, TensorData, SimulationParams, PrecalcalculatedTerms

In [12]:
state = state**2
p = state
i = 1
a = p - i

t_a_i_tensor = np.full((a), 0, dtype=complex)
t_ab_ij_tensor = np.full((site, a, a), 0, dtype=complex)

#eigenvalues from h for update
epsilon = np.diag(h_full)

params = SimulationParams(
a=a,
i=i,
p=p,  # These can be the same as `a + i` or chosen independently
site=site,
state=state,
i_method=3,
gap=False,
gap_site=3,
epsilon=epsilon,
periodic=True
)

tensors = TensorData(
t_a_i_tensor=t_a_i_tensor,
t_ab_ij_tensor=t_ab_ij_tensor,
h_full=h_full,
v_full=v_full
)

terms = PrecalcalculatedTerms()
qs_new = QuantumSimulation(params, tensors, terms)

# Initialize T_0 (reference amplitude) as complex zero
t_0 = complex(0)
# Initialize T_ai amplitudes as zeros
single = np.zeros((a), dtype = complex)
double = np.zeros((site, a, a), dtype = complex)

qs_new.terms.h_pp=qs_new.h_term(p, p)
qs_new.terms.h_pa=qs_new.h_term(p, a)
qs_new.terms.h_ip=qs_new.h_term(i, p).reshape(p)
qs_new.terms.V_pppp=qs_new.v_term(p, p, p, p, 0, 1).reshape(p**2, p**2)
qs_new.terms.V_ppaa=qs_new.v_term(p, p, a, a, 0, 1).reshape(p**2, a**2)
qs_new.terms.V_iipp=qs_new.v_term(i, i, p, p, 0, 1).reshape(p, p)
qs_new.terms.V_iiaa=qs_new.v_term(i, i, a, a, 0, 1).reshape(a, a)
qs_new.terms.V_piaa=qs_new.v_term(p, i, a, a, 0, 1).reshape(p, a**2)
qs_new.terms.V_pipp=qs_new.v_term(p, i, p, p, 0, 1).reshape(p, p**2)
qs_new.terms.V_ipap=qs_new.v_term(i, p, a, p, 0, 1).reshape(p, a, p)
qs_new.terms.V_piap=qs_new.v_term(p, i, a, p, 0, 1).reshape(p, a, p)

In [13]:
site, a, p, i = params.site, params.a, params.p, params.i

print(params.a)

dTab_ijdB_sol_new, dTa_idB_sol_new, T_ai_new = comb_flat[:-a-1], comb_flat[-a-1:-1], comb_flat[-1] 
dTab_ijdB_new = dTab_ijdB_sol_new.reshape(site, a, a)
dTa_idB_new = dTa_idB_sol_new.reshape(a)

qs_new.tensors.t_a_i_tensor = dTa_idB_new

for site_1 in range(1, site):
    qs_new.tensors.t_ab_ij_tensor[site_1] = dTab_ijdB_new[site_1]
    print(dTab_ijdB_new[site_1])

# print(dTab_ijdB_sol_new.reshape(site, a, a))

two_max = qs_new.tensors.t_ab_ij_tensor.flat[np.argmax(np.abs(qs_new.tensors.t_ab_ij_tensor))]

qs_new.terms.a_term=qs_new.A_term(a)
qs_new.terms.b_term=qs_new.B_term(i)
qs_new.terms.bb_term=oe.contract("q,s->qs", qs_new.terms.b_term, qs_new.terms.b_term).reshape(p**2)
qs_new.terms.aa_term=oe.contract("ap,bq->abpq", qs_new.terms.a_term, qs_new.terms.a_term).reshape(a**2, p**2)

energy_new = 0

for site_x in range(site):
    energy_new += qs_new.terms.h_ip @ qs_new.terms.b_term

    for site_y in range(site_x + 1, site_x + site):
        if abs(site_x - site_y) == 1 or abs(site_x - site_y) == (site - 1):
            V_iipp = qs_new.terms.V_iipp
            V_iiaa = qs_new.terms.V_iiaa
            T_xy = qs_new.t_term(site_x, site_y)

            # noinspection SpellCheckingInspection
            energy_new +=  np.sum(V_iiaa* (T_xy)) * 0.5
            # noinspection SpellCheckingInspection
            energy_new += (V_iipp @ qs_new.terms.b_term @ qs_new.terms.b_term) * 0.5

single_new = np.zeros((a), dtype = complex)
double_new = np.zeros((site, a, a), dtype = complex)

single_new = qs_new.residual_single()
for y_site in range(1, site):
    double_new[y_site] = qs_new.residual_double_total(y_site)

dTab_ijdB_new = (-1*(double_new))
dTa_idB_new = (-1*(single))
dT_0dB_new = [-1*(energy_new)]

8
[[0.04303841 0.75613205 0.14282747 0.44720885 0.6343892  0.45678408 0.58477001 0.03802739]
 [0.33330926 0.73634529 0.67561855 0.70120182 0.40067707 0.13875329 0.77138087 0.70709578]
 [0.44076556 0.40888282 0.68900302 0.32141623 0.13982263 0.62033259 0.20993148 0.12559231]
 [0.19496029 0.82959488 0.97146558 0.8708089  0.50659388 0.71835095 0.80456471 0.55204693]
 [0.78754031 0.38153515 0.66530756 0.10236386 0.40554834 0.76857719 0.8731257  0.95287714]
 [0.12819184 0.94321439 0.17254984 0.37483241 0.71857159 0.26932257 0.05598014 0.00497767]
 [0.09892333 0.56265874 0.25641306 0.75715053 0.37060377 0.77031252 0.12665693 0.4833955 ]
 [0.47218835 0.10634449 0.18326103 0.27869497 0.50130196 0.16570829 0.0690733  0.08698876]]
[[0.72365431 0.36165942 0.53059984 0.99952927 0.84114737 0.03315338 0.99208089 0.79940207]
 [0.42273196 0.99285412 0.35433084 0.74436735 0.67871641 0.8210294  0.647144   0.0904609 ]
 [0.29394214 0.1921458  0.56514068 0.78040349 0.13037644 0.84452554 0.06668477 0.876890

In [14]:
qs_new.terms.b_term

array([1.        , 0.61906469, 0.41144321, 0.55913943, 0.08587843, 0.24916508, 0.62061379, 0.35890265, 0.62627943])

## Old.

In [15]:
from quant_rotor.models.dense.t_amplitudes_sub_class import QuantumSimulation, TensorData, SimulationParams

In [16]:
site = 3
state = 3
g = 0.2

state = state**2
p = state
i = 1
a = p - i

In [17]:
t_a_i_tensor = np.full((site, a, i), 0, dtype=complex)
t_ab_ij_tensor = np.full((site, site, a, a, i, i), 0, dtype=complex)

#eigenvalues from h for update
epsilon = np.diag(h_full)

params = SimulationParams(
a=a,
i=i,
p=p,  # These can be the same as `a + i` or chosen independently
site=site,
state=state,
i_method=3,
gap=False,
gap_site=3,
epsilon=epsilon,
periodic=True
)

tensors = TensorData(
t_a_i_tensor=t_a_i_tensor,
t_ab_ij_tensor=t_ab_ij_tensor,
h_full=h_full,
v_full=v_full
)

qs_old = QuantumSimulation(params, tensors)

# Initialize T_0 (reference amplitude) as complex zero
t_0 = complex(0)
# Initialize T_ai amplitudes as zeros
single = np.zeros((a,i), dtype = complex)
double = np.zeros((site, a, a, i, i), dtype = complex)

In [18]:
qs_old.h_term(i, p) - qs_new.terms.h_ip

array([[0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [19]:
qs_old.B_term(i, site_x).reshape(p) - qs_new.terms.b_term

array([ 0.        +0.j, -0.61906469+0.j, -0.41144321+0.j, -0.55913943+0.j, -0.08587843+0.j, -0.24916508+0.j, -0.62061379+0.j, -0.35890265+0.j, -0.62627943+0.j])

In [20]:
site, a, p, i = params.site, params.a, params.p, params.i

dTab_ijdB_sol_old, dTa_idB_sol_old, T_ai_old = comb_flat[:-a-1], comb_flat[-a-1:-1], comb_flat[-1] 
dTab_ijdB_old = dTab_ijdB_sol_old.reshape(site, a, a, i, i)
dTa_idB_old = dTa_idB_sol_old.reshape(a, i)

# print(dTab_ijdB_sol_old.reshape(site, a, a, i, i).reshape(site, a, a))

# print(np.array_equal(dTab_ijdB_sol_old.reshape(site, a, a), dTab_ijdB_sol_new.reshape(site, a, a)))
# print(np.array_equal(dTa_idB_sol_old.reshape(a), dTa_idB_sol_new.reshape(a)))

qs_old.tensors.t_a_i_tensor[0] = dTa_idB_old

for site_1 in range(1, site):
    qs_old.tensors.t_a_i_tensor[site_1] = qs_old.tensors.t_a_i_tensor[0]
    # print("t_1:", np.array_equal(qs_old.tensors.t_a_i_tensor[0].reshape(a), qs_new.tensors.t_a_i_tensor))
    qs_old.tensors.t_ab_ij_tensor[0, site_1] = dTab_ijdB_old[site_1]
    for site_2 in range(1, site):
        qs_old.tensors.t_ab_ij_tensor[site_2, (site_1 + site_2) % site] = qs_old.tensors.t_ab_ij_tensor[0, site_1]
        # print("t_2:", np.array_equal(qs_old.tensors.t_ab_ij_tensor[site_2, (site_1 + site_2) % site].reshape(a, a), qs_new.tensors.t_ab_ij_tensor[site_1]))
    
print("t_2:", np.array_equal(qs_old.tensors.t_ab_ij_tensor[0].reshape(site, a, a), qs_new.tensors.t_ab_ij_tensor))

two_max = qs_old.tensors.t_ab_ij_tensor.flat[np.argmax(np.abs(qs_old.tensors.t_ab_ij_tensor))]

energy_old = 0

for site_x in range(site):
    energy_old += np.einsum("ip, pi->", qs_old.h_term(i, p), qs_old.B_term(i, site_x))

    for site_y in range(site_x + 1, site_x + site):
        # noinspection SpellCheckingInspection
        energy_old += np.einsum("ijab, abij->", qs_old.v_term(i, i, a, a, site_x, site_y % site), qs_old.t_term(site_x, site_y % site)) * 0.5
        # noinspection SpellCheckingInspection
        energy_old += np.einsum("ijpq, pi, qj->", qs_old.v_term(i, i, p, p, site_x, site_y % site), qs_old.B_term(i, site_x), qs_old.B_term(i, site_y % site)) * 0.5

print("Energy:", np.array_equal(energy_new, energy_old))

single_old = np.zeros((a,i), dtype = complex)
double_old = np.zeros((site, a, a, i, i), dtype = complex)

single_old = qs_old.residual_single(0)

print("Singles:", np.allclose(single_old.reshape(a), single_new))
print(single_old.reshape(a) - single_new)

for y_site in range(1, site):
    double_old[y_site] = qs_old.residual_double_total(0, y_site)
    print("Double:", np.allclose(qs_old.residual_double_total(0, y_site).reshape(a, a), qs_new.residual_double_total(y_site)))
    print("Double:", qs_old.residual_double_total(0, y_site).reshape(a, a) - qs_new.residual_double_total(y_site))

dTa_idB_old = (-1*(single_old))
dTab_ijdB_old = (-1*(double_old))
dT_0dB_old = [-1*(energy_old)]

t_2: True
Energy: True
Singles: True
[ 1.94289029e-16+0.j  2.77555756e-17+0.j  1.11022302e-16+0.j  6.93889390e-17+0.j  6.24500451e-17+0.j -1.11022302e-16+0.j  1.38777878e-17+0.j  1.11022302e-16+0.j]
Double: True
Double: [[ 1.66533454e-16+0.j  1.11022302e-16+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j  1.11022302e-16+0.j -1.38777878e-17+0.j  1.11022302e-16+0.j  1.11022302e-16+0.j]
 [ 1.11022302e-16+0.j  2.22044605e-16+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j  1.11022302e-16+0.j  5.55111512e-17+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [-2.22044605e-16+0.j  0.00000000e+00+0.j  4.44089210e-16+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j  4.44089210e-16+0.j  0.00000000e+00+0.j -1.11022302e-16+0.j]
 [ 5.55111512e-17+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j -2.22044605e-16+0.j  0.00000000e+00+0.j]
 [ 2.77555756e-16+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j  2.77555756e-16+0.j  8.32667268e-17+0.j  5.55111512e-17+0.j  2.220

In [21]:
print(single_new)
double_new

[-0.03155283+0.j -0.03473602+0.j -0.56987016+0.j  0.11820631+0.j  0.01818648+0.j -0.27592288+0.j -0.00949698+0.j -0.66062939+0.j]


array([[[ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j],
        [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j],
        [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j],
        [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j],
        [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j],
        [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0

In [22]:
print(single_old.reshape(a))
double_old.reshape(site, a, a)

[-0.03155283+0.j -0.03473602+0.j -0.56987016+0.j  0.11820631+0.j  0.01818648+0.j -0.27592288+0.j -0.00949698+0.j -0.66062939+0.j]


array([[[ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j],
        [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j],
        [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j],
        [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j],
        [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j],
        [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0

In [23]:
print(np.array_equal(dTab_ijdB_new, dTa_idB_old))
print(np.array_equal(dTab_ijdB_new, dTab_ijdB_old))
print(np.array_equal(dT_0dB_new, dT_0dB_old))

False
False
True
